In [30]:
import psycopg2
import csv
import sys
import codecs
import csv
import numpy
import networkx as nx
import itertools
from nltk.corpus import stopwords
from nltk.corpus import words
from sklearn import cross_validation
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer


conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="secret")
cur = conn.cursor()


stopWords = set(stopwords.words('english'))
allWords = set(words.words())


rankL = list()
songL = list()
artistL = list()
yearL = list()
lyricsL = list()
topQuartile = list()

cur.execute('''
SELECT rank, song, artist, year, lyrics
FROM songs
WHERE year < 1990
ORDER BY id;
'''
)

for rank, song, artist, year, lyrics in cur:
    rankL.append(rank)
    songL.append(song)
    artistL.append(artist)
    yearL.append(year)
    lyricswords = lyrics.split()
    resultwords  = [word for word in lyricswords if (word.lower() in allWords and word.lower() not in stopWords)]
    result = ' '.join(resultwords)
    lyricsL.append(result)
    topQuartile.append(1 if rank <= 25 else 0)


vectorizer = CountVectorizer(binary=True, stop_words=None)
vectorizer.fit(lyricsL)
lyric_features = vectorizer.get_feature_names()
X = vectorizer.transform(lyricsL)

print('the' in lyric_features)




False


In [3]:
noInstances = format(X.shape[0])
print '{} instances'.format(X.shape[0])
print '{} features'.format(X.shape[1])

500 instances
4309 features


### Naive Bayes Test Accuracy 2010-2015

In [4]:
X_train, X_test, rank_train, rank_test = cross_validation.train_test_split(X, topQuartile, test_size=0.33, random_state=216)

In [5]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train.toarray(), numpy.asarray(rank_train))
print 'train accuracy: {:.4f}'.format(mnb.score(X_train.toarray(), numpy.asarray(rank_train)))
print 'test accuracy: {:.4f}'.format(mnb.score(X_test.toarray(), numpy.asarray(rank_test)))

train accuracy: 0.9254
test accuracy: 0.7152


In [6]:
n = 100
topWords = list()
coefs_with_fns = sorted(zip(mnb.coef_[0], lyric_features))
top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
for (coef_1, fn_1), (coef_2, fn_2) in top:
    print "\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2)
    topWords.append(fn_2)

	-9.2045	abbey          		-5.1792	dont           
	-9.2045	abbot          		-5.2342	like           
	-9.2045	able           		-5.2727	see            
	-9.2045	abroad         		-5.3127	love           
	-9.2045	absurd         		-5.3333	know           
	-9.2045	academy        		-5.3979	got            
	-9.2045	accent         		-5.4433	oh             
	-9.2045	accept         		-5.4433	get            
	-9.2045	acceptance     		-5.4910	cause          
	-9.2045	account        		-5.4910	cant           
	-9.2045	accountant     		-5.6782	go             
	-9.2045	ache           		-5.7080	one            
	-9.2045	aching         		-5.7388	never          
	-9.2045	acrobat        		-5.7388	come           
	-9.2045	act            		-5.7388	back           
	-9.2045	actin          		-5.7705	yeah           
	-9.2045	action         		-5.7705	baby           
	-9.2045	active         		-5.8033	time           
	-9.2045	addition       		-5.8033	tell           
	-9.2045	admiring       		-5.8033	say            


In [23]:
G = nx.Graph(name="lyrics")

G.add_nodes_from(topWords)

for a,b in itertools.combinations(topWords,2):
    if(numpy.sum((X[:,lyric_features.index(a)] + X[:,lyric_features.index(b)]) == 2) > 50):
        G.add_edge(a,b)
            
    

In [15]:
print("Graph has %d nodes with %d edges"
          % (nx.number_of_nodes(G), nx.number_of_edges(G)))

Graph has 100 nodes with 865 edges


In [24]:
nx.write_gexf(G, '2010lyrics.gexf')

### Naive Bayes Test Accuracy 1964-2015

In [27]:
X_train, X_test, rank_train, rank_test = cross_validation.train_test_split(X, topQuartile, test_size=0.33, random_state=216)

In [28]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train.toarray(), numpy.asarray(rank_train))
print 'train accuracy: {:.4f}'.format(mnb.score(X_train.toarray(), numpy.asarray(rank_train)))
print 'test accuracy: {:.4f}'.format(mnb.score(X_test.toarray(), numpy.asarray(rank_test)))

train accuracy: 0.8338
test accuracy: 0.7166


In [29]:
n = 100
topWords = list()
coefs_with_fns = sorted(zip(mnb.coef_[0], lyric_features))
top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
for (coef_1, fn_1), (coef_2, fn_2) in top:
    print "\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2)
    topWords.append(fn_2)

	-10.9551	aa             		-4.7465	know           
	-10.9551	aback          		-4.7834	love           
	-10.9551	abandon        		-4.7876	dont           
	-10.9551	abandoned      		-4.9077	like           
	-10.9551	abbot          		-5.0718	see            
	-10.9551	abbreviation   		-5.0746	got            
	-10.9551	abdicate       		-5.1116	get            
	-10.9551	abdominal      		-5.1590	oh             
	-10.9551	abduction      		-5.1774	baby           
	-10.9551	abide          		-5.1899	time           
	-10.9551	ability        		-5.1962	cause          
	-10.9551	abortion       		-5.2153	cant           
	-10.9551	abounding      		-5.2250	say            
	-10.9551	abroad         		-5.2414	make           
	-10.9551	abruptly       		-5.2447	one            
	-10.9551	absence        		-5.2547	go             
	-10.9551	absorb         		-5.2715	never          
	-10.9551	absorbed       		-5.3132	way            
	-10.9551	absurd         		-5.3239	come           
	-10.9551	abut           		-5.3

### Naive Bayes Test Accuracy 1964-1990

In [31]:
X_train, X_test, rank_train, rank_test = cross_validation.train_test_split(X, topQuartile, test_size=0.33, random_state=216)

In [32]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train.toarray(), numpy.asarray(rank_train))
print 'train accuracy: {:.4f}'.format(mnb.score(X_train.toarray(), numpy.asarray(rank_train)))
print 'test accuracy: {:.4f}'.format(mnb.score(X_test.toarray(), numpy.asarray(rank_test)))

train accuracy: 0.8382
test accuracy: 0.7115


In [33]:
n = 100
topWords = list()
coefs_with_fns = sorted(zip(mnb.coef_[0], lyric_features))
top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
for (coef_1, fn_1), (coef_2, fn_2) in top:
    print "\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2)
    topWords.append(fn_2)

	-10.0954	aa             		-4.7109	love           
	-10.0954	aback          		-4.8021	know           
	-10.0954	abandon        		-4.9537	dont           
	-10.0954	abandoned      		-5.1256	like           
	-10.0954	abdicate       		-5.1256	got            
	-10.0954	ability        		-5.1325	oh             
	-10.0954	able           		-5.1901	see            
	-10.0954	aboard         		-5.2356	never          
	-10.0954	abounding      		-5.2356	cant           
	-10.0954	absence        		-5.2512	time           
	-10.0954	absurd         		-5.2832	get            
	-10.0954	abut           		-5.2832	baby           
	-10.0954	accent         		-5.2996	way            
	-10.0954	acceptable     		-5.2996	one            
	-10.0954	accomplished   		-5.3079	say            
	-10.0954	account        		-5.3418	make           
	-10.0954	accurate       		-5.3859	go             
	-10.0954	accusation     		-5.3859	come           
	-10.0954	accuse         		-5.4319	yeah           
	-10.0954	accused        		-5.4

In [26]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u"you're", u"you've", u"you'll", u"you'd", u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u"she's", u'her', u'hers', u'herself', u'it', u"it's", u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u"that'll", u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'eac